# Guide to `deph.utils` Utilities

The `deph` package includes a powerful and dependency-free utility module named `devh`. This module provides simple solutions for common development tasks.

This notebook demonstrates the three core components of `devh`:

1.  **`deph.utils.log`**: A simple, flexible logging setup utility.
2.  **`deph.utils.pip`**: A programmatic wrapper for `pip` to install, query, and manage packages.
3.  **`deph.utils.zip`**: A lightweight toolkit for creating and inspecting ZIP archives.

## 1. `deph.utils.log` - Simplified Logging

`deph.utils.log` lets you configure console and file logging with a single function call, avoiding the usual boilerplate.

In [5]:
import sys
import logging
import os
from pathlib import Path
module_dir = Path.cwd().parent / 'src'
sys.path.append(module_dir.as_posix())
from deph.utils import log

# --- Example 1: Basic Console Logging ---
print("--- Console Logging ---")
log.init(level=logging.INFO) # Set up logging to the console
log.emit("This is an info message.")
log.emit("This is a warning.", level="warning")
log.emit("This debug message won't be shown.", level="debug")

# --- Example 2: Logging to a File ---
LOG_FILE = Path("test_app.log")
log.init(log_file=LOG_FILE, use_console=False, level=logging.DEBUG)
log.emit("This message goes only to the file.", level="debug")

print(f"\n--- File Logging (Contents of {LOG_FILE}) ---")
print(LOG_FILE.read_text().strip())
os.remove(LOG_FILE)

--- Console Logging ---
This is an info message.


[WARNING ] __main__: This is a warning.



--- File Logging (Contents of test_app.log) ---
2025-10-05 13:15:46,226 [DEBUG   ]  __main__: This message goes only to the file.


## 2. `deph.utils.pip` - Programmatic Package Management

`deph.utils.pip` provides a safe and reliable way to call `pip` from your Python scripts, returning structured JSON data instead of plain text.

In [7]:
from deph.utils import pip as util_pip
from pprint import pprint

if util_pip is None:
    print("pip utilities are unavailable in this environment.")
else:
    # Initialize the wrapper for the current Python environment
    installer = util_pip.Pip()

    # --- Example 1: Install, Check, and Uninstall a Package ---
    PACKAGE_NAME = "tqdm"

    print(f"--- 1. Checking if '{PACKAGE_NAME}' is installed ---")
    installed_version = installer.is_installed(PACKAGE_NAME)
    print(f"Is '{PACKAGE_NAME}' installed? {'Yes, version ' + installed_version if installed_version else 'No'}")

    if not installed_version:
        try:
            print(f"\n--- 2. Installing '{PACKAGE_NAME}' ---")
            rep = installer.install(PACKAGE_NAME)
            print("Installed.")
        except Exception as e:
            print(f"Install failed: {e}")

    # --- Optional: Uninstall (commented out) ---
    # try:
    #     print(f"\n--- 3. Uninstalling '{PACKAGE_NAME}' ---")
    #     rep_un = installer.uninstall(PACKAGE_NAME)
    #     print("Uninstalled.")
    # except Exception as e:
    #     print(f"Uninstall failed: {e}")


--- 1. Checking if 'tqdm' is installed ---
Is 'tqdm' installed? Yes, version 4.67.1


In [8]:
# --- Example 2: List available versions of a package ---
print("\n--- Available versions for 'pandas' (latest 5) ---")
if util_pip is None:
    print("pip utilities are unavailable in this environment.")
else:
    try:
        pandas_versions = installer.available_versions("pandas")
        pprint(pandas_versions[:5])
    except Exception as e:
        print(f"Could not fetch versions. Maybe you are offline? Error: {e}")



--- Available versions for 'pandas' (latest 5) ---
['2.3.3', '2.3.2', '2.3.1', '2.3.0', '2.2.3']


## 3. `deph.utils.zip` - ZIP Archive Handling

`deph.utils.zip` is a utility for creating and inspecting ZIP files without needing external libraries. It's perfect for packaging artifacts or analyzing existing archives.

In [9]:
from deph.utils import zip

# --- Setup: Create some dummy files and directories ---
Path("my_archive").mkdir(exist_ok=True)
Path("my_archive/data.csv").write_text("col1,col2\n1,2")
Path("my_archive/config").mkdir(exist_ok=True)
Path("my_archive/config/settings.json").write_text('{"key": "value"}')

ZIP_PATH = Path("archive.zip")

# --- Example 1: Create a ZIP from a directory ---
print(f"--- Creating {ZIP_PATH} from 'my_archive' directory ---")
zip.create_from_dir(ZIP_PATH, "my_archive")
print(f"Archive created: {ZIP_PATH.exists()}")

# --- Example 2: Walk through the contents of the ZIP ---
print("\n--- Contents of the ZIP archive ---")
for root, dirs, files in zip.walk(zip.load(ZIP_PATH)):
    print(f"Directory: {root}")
    if dirs:
        print(f"  Sub-directories: {dirs}")
    if files:
        print(f"  Files: {files}")

# --- Cleanup ---
import shutil
os.remove(ZIP_PATH)
shutil.rmtree("my_archive")
print("\n--- Cleanup complete ---")

--- Creating archive.zip from 'my_archive' directory ---
Archive created: True

--- Contents of the ZIP archive ---
Directory: 
  Sub-directories: ['config']
  Files: [ZippedFile(name='data.csv', arcname='data.csv', zipobj=<zipfile.ZipFile filename='archive.zip' mode='r'>)]
Directory: config
  Files: [ZippedFile(name='settings.json', arcname='config/settings.json', zipobj=<zipfile.ZipFile filename='archive.zip' mode='r'>)]

--- Cleanup complete ---
